# BPE 구현하기

### 실습 목차
* 1. Tokenization
  * 1-1. Subword Segmentation
  * 1-2. Byte-Pair Encoding
* 2. BPE 알고리즘을 활용한 한국어 토크나이저 만들기
  * 2-1. Pre-tokenization
  * 2-2. BPE 구현하기
    * 2-2-1. `get_vocab` 함수
    * 2-2-2. `get_stats` 함수
    * 2-2-3. `merge_vocab` 함수
  * 2-3. BPE 적용하기

### 1. Tokenization
- 자연어 처리에서 코퍼스 데이터를 데이터를 사용하고자 하는 용도에 맞게 토큰화 & 정제 & 정규화라는 일을 하게 됩니다.
- 주어진 코퍼스에서 입력 시퀀스를 토큰이라 불리는 단위로 나누는 작업을 토큰화라고 합니다.
- 토큰의 기준을 정의하는 것에 따라 토큰화의 결과가 달라지게 됩니다.
- 그 중, 서브워드 분리(Subword segmentation) 알고리즘인 **BPE (Byte-Pair Encoding)**에 대해서 살펴보겠습니다.

#### 1-1. 서브워드 분리 알고리즘 (Subword segmentation)
- 기계가 문제를 풀 때 모르는 단어가 등장하면 (사람도 마찬가지지만) 주어진 문제를 푸는 것이 까다로워집니다. 이와 같이 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황을 OOV(Out-Of-Vocabulary) 문제라고 합니다.
- 서브워드 분리 작업은 ```하나의 단어를 더 작은 단위의 의미있는 여러 하위 단어들로 분리```해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업입니다.
- 예를 들어, "패스트캠퍼스"라는 단어는 "패스트"와 "캠퍼스"라는 단어로 분리될 수 있습니다.
- 이를 통해, OOV나 희귀 단어, 신조어와 같은 문제를 완화시킬 수 있습니다.

#### 1-2. Byte-Pair Encoding
- 서브워드 분리 알고리즘의 한 종류로서, 글자 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용합니다.

- BPE는 원래 정보를 압축하는 알고리즘입니다. 데이터에서 가장 많이 등장한 문자열을 병합해서 데이터를 압축하는 기법입니다. 최근에는 자연어 처리 모델에서 토큰화 기법으로 널리 쓰이고 있습니다.

- BPE를 활용한 토크나이즈 절차는 다음과 같습니다.
1. **어휘 집합 구축**: ```자주 등장하는 문자열(서브워드)을 하나의 글자로 병합```한 뒤 어휘 집합에 추가합니다. 이를 ```원하는 어휘 집합 크기가 될 때까지 반복```합니다.
2. **토큰화**: 토큰화 대상 문장 내 각 어절(띄어쓰기로 문장을 나눈 것)에서 어휘 집합에 있는 서브워드가 포함되어 있을 때 해당 서브워드를 어절에서 분리합니다.

#### 어휘 집합 구축 과정
1. 초기 어휘 집합 구축 (단어 빈도)
2. 글자 단위로 분리 후, 문자열 별 빈도 카운트
3. 최빈도 문자열 어휘 집합에 추가
4. 문자열 별 빈도 카운트 업데이트
5. 다시 3번부터 반복




### 2. BPE 알고리즘을 활용한 한국어 토크나이저 만들기
BPE 어휘 집합 구축 절차를 구체적으로 살펴보겠습니다.

#### 2-1. Pre-tokenization
- 어휘 집합을 만들기 위해선 준비된 말뭉치로부터 프리토크나이즈 작업을 수행합니다. 말뭉치의 모든 문장을 상황에 맞는 기준으로 전처리 합니다.

- BPE 알고리즘 기반의 한국어 토크나이저가 어떻게 작동하는지 간단하게 확인하기 함이 목적이기 때문에 불필요한 특수 문자와 숫자, 영어를 모두 제거할 수 있도록 합니다.


In [ ]:
import re
from collections import defaultdict
from typing import Dict, List, Tuple
from tqdm import tqdm

corpus = [
    "자연어처리를 사랑해",
    "인공지능을 사랑해",
    "머신러닝을 사랑해",
    "공부는 사랑하지 않아",
    "게임을 사랑하지 않아",
    "내가 사랑하는 너",
    "네가 사랑하는 나"
]


SPECIALS = "".join([".", ",", ";", ":", "!", "?", '"', "'", " "])


def preprocess(text: str, only_kor: bool=True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|{SPECIALS}|a-zA-Z|]+", "", text)

    # 연속 공백 제거
    text = re.sub(" +", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()

#### 2-2. BPE 구현하기

- BPE 알고리즘은 다음과 같은 3개의 주요 함수로 이루어집니다.
    1. `get_vocab` <br>
    : 토큰을 글자 단위로 분리한 후 어휘 사전에 등록합니다.
    2. `get_stats` <br>
    : 문자열 출현 빈도를 구하여 pair를 출력합니다.
    3. `merge_vocab` <br>
    : vocab에 존재하는 두 문자열를 새로운 글자로 병합하는 규칙을 통해 원하는 vocab의 크기에 도달할 때까지 새 어휘 추가합니다. <br>
    : 따라서, 처음에는 이러한 병합으로 두 개의 글자가 있는 문자열이 생성되고 학습이 점점 진행됨에 따라 더 긴 문자열(서브워드)이 생성됩니다. <br>
    : merges 과정에서 BPE 알고리즘은 가정 빈번하게 출현하는 문자열을 검색합니다.

##### 2-2-1. `get_vocab` 함수
    1. 코퍼스 파일을 라인 단위로 읽어옵니다.
    2. 각 라인을 공백 단위로 나누어 토큰 리스트를 구성합니다.
    3. 토큰 리스트를 순회하며, 토큰을 캐릭터 단위로 자른 후 사전에 postfix </w>와 함께 등록합니다.
      - </w> 토큰 : 단어 중간에 나오는 token인지, 단어 마지막에 나오는 token인지 구분하기 위함.

In [ ]:
def get_vocab(f_name: str) -> Dict[str, int]:
    """코퍼스 파일을 읽어와 단어 사전 구축"""
    vocab = defaultdict(int)

    for line in corpus:
        tokens = preprocess(line).strip().split()
        for token in tokens:
            vocab[" ".join(list(token)) + "</w>"] += 1
    return dict(vocab)

##### 2-2-2. `get_stats` 함수

In [ ]:
def get_stats(vocab: Dict[str, int]):
    """사전을 활용한 바이그램 pair 구축"""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return dict(pairs)

##### 2-2-3. `merge_vocab` 함수
    1. get_stats 함수를 통해 얻어진 바이그램 중 가장 자주 등장한 pair와 기존 사전을 인자로 받습니다.
    2. 기존 사전에서 캐릭터 단위로 떨어져 있던 바이그램을 하나의 pair로 엮어줍니다.
    3. 새로 구축한 어휘 사전을 반환합니다.

In [ ]:
def merge_vocab(pair: Tuple[str, str], vocab: Dict[str, int]):
    """가장 자주 등장한 바이그램 페어를 엮어줌"""
    result = defaultdict(int)
    for word in vocab:
        paired = word.replace(" ".join(pair), "".join(pair))
        result[paired] = vocab[word]
    return dict(result)

#### 2-3 BPE 적용하기
- 지금까지 구현한 함수들을 활용해 자주 등장한 Byte-Pair가 합쳐져, 새로운 사전이 구축되는 과정을 살펴봅니다. <br>
- `num_merges`를 설정하여 사전을 합치는 횟수를 정해줍니다. 해당 인자에 따라 사전의 전체 크기를 조정할 수 있습니다.

In [ ]:
vocab = get_vocab(corpus) ## 한글자 단위로 분할.
vocab

{'자 연 어 처 리 를</w>': 1,
 '사 랑 해</w>': 3,
 '인 공 지 능 을</w>': 1,
 '머 신 러 닝 을</w>': 1,
 '공 부 는</w>': 1,
 '사 랑 하 지</w>': 2,
 '않 아</w>': 2,
 '게 임 을</w>': 1,
 '내 가</w>': 1,
 '사 랑 하 는</w>': 2,
 '너</w>': 1,
 '네 가</w>': 1,
 '나</w>': 1}

In [ ]:
# 구축된 어휘 사전을 순회하며, 사전 내 등록된 캐릭터 토큰과 등장 횟수를 반환합니다.

def get_tokens(vocab: Dict[str, int]):
    """사전 내 등록된 토큰을 확인"""
    result = defaultdict(int)
    for word, freq in vocab.items():
        tokens = word.split()
        for token in tokens:
            result[token] += freq
    return dict(result)

In [ ]:
tokens = get_tokens(vocab)
tokens

{'자': 1,
 '연': 1,
 '어': 1,
 '처': 1,
 '리': 1,
 '를</w>': 1,
 '사': 7,
 '랑': 7,
 '해</w>': 3,
 '인': 1,
 '공': 2,
 '지': 1,
 '능': 1,
 '을</w>': 3,
 '머': 1,
 '신': 1,
 '러': 1,
 '닝': 1,
 '부': 1,
 '는</w>': 3,
 '하': 4,
 '지</w>': 2,
 '않': 2,
 '아</w>': 2,
 '게': 1,
 '임': 1,
 '내': 1,
 '가</w>': 2,
 '너</w>': 1,
 '네': 1,
 '나</w>': 1}

In [ ]:
pairs = get_stats(vocab)
pairs

{('자', '연'): 1,
 ('연', '어'): 1,
 ('어', '처'): 1,
 ('처', '리'): 1,
 ('리', '를</w>'): 1,
 ('사', '랑'): 7,
 ('랑', '해</w>'): 3,
 ('인', '공'): 1,
 ('공', '지'): 1,
 ('지', '능'): 1,
 ('능', '을</w>'): 1,
 ('머', '신'): 1,
 ('신', '러'): 1,
 ('러', '닝'): 1,
 ('닝', '을</w>'): 1,
 ('공', '부'): 1,
 ('부', '는</w>'): 1,
 ('랑', '하'): 4,
 ('하', '지</w>'): 2,
 ('않', '아</w>'): 2,
 ('게', '임'): 1,
 ('임', '을</w>'): 1,
 ('내', '가</w>'): 1,
 ('하', '는</w>'): 2,
 ('네', '가</w>'): 1}

In [ ]:
num_merges = 5

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    tokens = get_tokens(vocab)
    print("Iter: ", i+1)
    print("Best pair: ", best)
    print("Tokens: ", tokens)
    print("Number of tokens: ", len(tokens))

Iter:  1
Best pair:  ('사', '랑')
Tokens:  {'자': 1, '연': 1, '어': 1, '처': 1, '리': 1, '를</w>': 1, '사랑': 7, '해</w>': 3, '인': 1, '공': 2, '지': 1, '능': 1, '을</w>': 3, '머': 1, '신': 1, '러': 1, '닝': 1, '부': 1, '는</w>': 3, '하': 4, '지</w>': 2, '않': 2, '아</w>': 2, '게': 1, '임': 1, '내': 1, '가</w>': 2, '너</w>': 1, '네': 1, '나</w>': 1}
Number of tokens:  30
Iter:  2
Best pair:  ('사랑', '하')
Tokens:  {'자': 1, '연': 1, '어': 1, '처': 1, '리': 1, '를</w>': 1, '사랑': 3, '해</w>': 3, '인': 1, '공': 2, '지': 1, '능': 1, '을</w>': 3, '머': 1, '신': 1, '러': 1, '닝': 1, '부': 1, '는</w>': 3, '사랑하': 4, '지</w>': 2, '않': 2, '아</w>': 2, '게': 1, '임': 1, '내': 1, '가</w>': 2, '너</w>': 1, '네': 1, '나</w>': 1}
Number of tokens:  30
Iter:  3
Best pair:  ('사랑', '해</w>')
Tokens:  {'자': 1, '연': 1, '어': 1, '처': 1, '리': 1, '를</w>': 1, '사랑해</w>': 3, '인': 1, '공': 2, '지': 1, '능': 1, '을</w>': 3, '머': 1, '신': 1, '러': 1, '닝': 1, '부': 1, '는</w>': 3, '사랑하': 4, '지</w>': 2, '않': 2, '아</w>': 2, '게': 1, '임': 1, '내': 1, '가</w>': 2, '너</w>': 1, '네': 1, '나</w>':